# Small to Slide

It's very difficult for a LLM to correctly embed a PPT slide based on text alone. Luckily we have multi-modal LLMs!

#### Goal
Ingest PPT slides using GPT 4o's embedding + Create a scheme to retrieve relevant slides

## Outline
- Setup (API keys, Milvus connection, data loading)
- Pre-processing (Embeddings, Schema creation)
- Ingestion
- Retrieval
- Retrieval Evaluation

### Setup

In [2]:
import os
import re
import asyncio
from typing import Iterator, Tuple
from openai import AsyncOpenAI
openai_api_key = os.environ.get("OPENAI_API_KEY")
zilliz_uri = os.environ.get("ZILLIZ_URI")
zilliz_api_key = os.environ.get("ZILLIZ_API_KEY")

### Pre-Processing

In [3]:
# Load text data, put them into a list
text_path = './llamaparse.md'
def markdown_to_string(file_path: str) -> str:
    try:
        with open(file_path, 'r') as file:
            markdown_string = file.read()
        return markdown_string
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except IOError:
        print(f"Error reading file: {file_path}")
        return None

markdown_string = markdown_to_string(text_path)
regex = r"<!---\s*Page\s*\d+\s*--->"
parts = re.split(regex, markdown_string)

# Do just 5 for checking whether this works
# TODO: Delete or Comment this out if you don't want to embed too many things
parts = parts[:5]

In [4]:
# Type alias for readability
Embedding = list[float]

# Get OpenAI client
client = AsyncOpenAI(api_key=openai_api_key)
async def get_embedding(input_text: str) -> Embedding:
    response = await client.embeddings.create(
        input=input_text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [5]:
# Run embedding concurrently.
# WARNING: There are ~40 pages = ~40 concurrent calls
tasks = [asyncio.create_task(get_embedding(page)) for page in parts]
results: list[Embedding] = await asyncio.gather(*tasks)
text_embedding_pairs: list[Tuple[str, Embedding]] = list(zip(parts, results))

#### PDF to Images

In [6]:
!ls

NVDA-Company-Overview.pdf llamaparse.md             output_images
README.md                 milvus-ingest.ipynb       ve


**Warning:**

Installing Poppler can be tedious. Make sure to
- Install
- Add `poppler` to PATH for your machine

In [5]:
from pdf2image import convert_from_path
import os
from tqdm import tqdm


# Specify the path to your PDF file
pdf_path = "NVDA-Company-Overview.pdf"

# Specify the output folder for images
output_folder = "output_images"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert PDF to images
images = convert_from_path(pdf_path)

# Save each page as an image
for i, image in tqdm(enumerate(images)):
    image.save(f"{output_folder}/page_{i+1}.jpg", "JPEG")

42it [00:03, 11.68it/s]


#### Image Embedding

We'll be embedding each powerpoint slide into image embeddings.

In [ ]:
# Embed Images


# Pydantic schema for Data chunks
# TODO: Keep the model here for reference



# Package this up into a list of Pydantic objects


## Ingestion

### Image referencing in Collection

### Create Collection Schema

In [ ]:
# Create Collection

# Create references to images in Zilliz collection metadata


# Create Zilliz collection schema


# Connect to Milvus


# Upload to collection


# TODO: Create references to slides in frontend app